---

- **程序简介：** 准备好待爬取企业股票代码清单（.xls格式），通过传入自定义参数即可实现灵活爬取巨潮资讯网上的企业公告！

- **使用指南：** 将Codelist_path、Output_path分别改为自己的存储路径，同时根据自己的需求自定义Category_A、SeDate_A、Search_key等参数，然后直接运行即可~（注意：除了“Step1：自定义参数”部分，其它地方请勿修改！）

- **版本号：** 巨潮资讯网企业公告自定义爬取_V2.1

- **修订时间：** 2022.10.23

- **发布者：** yxj

---


# Part1：准备工作

> ------------------------------------使用说明：新手复用此代码时仅需要更改 **“Step1：自定义参数”** 中的几项项参数即可-----------------------------------------

## Step1：自定义参数

In [1]:
# 1、定义股票代码列表 和 爬取结果输出地址(最好指定一个空的文件夹)———必选项

Codelist_path = r'c:\Users\qq264\Desktop\SZ_exchange_crawler-master\data table\stock number input.xls'
Output_path = r'C:\Users\qq264\Desktop\SZ_exchange_crawler-master\data table\output'


# 此路径是Step4中的stock_list文件输出地址，若不执行Step4则无需修改
Stocklist_path = r'C:\Users\qq264\Desktop\SZ_exchange_crawler-master\data table\stock output.xls'

In [2]:
# 2、定义要爬取的报表类型（若同时爬取多项中间用;连接）———必选项

# 公司年度报告：category_ndbg_szsh
# 公司治理报告：category_gszl_szsh

Category_All = "category_ndbg_szsh;category_sjdbg_szsh;category_dshgg_szsh;category_rcjy_szsh;category_sf_szsh;category_pg_szsh;category_kzzq_szsh;category_bcgz_szsh;category_tbclts_szsh;category_tszlq_szsh;category_cqdq_szsh;category_qtrz_szsh;category_jj_szsh;category_zf_szsh;category_gszl_szsh;category_jshgg_szsh;category_yjygjxz_szsh;category_bndbg_szsh;category_yjdbg_szsh;category_qyfpxzcs_szsh;category_gddh_szsh;category_zj_szsh;category_gqjl_szsh;category_gszq_szsh;category_gqbd_szsh;category_fxts_szsh;"

# Category_A = Category_All

# Category_A = "category_ndbg_szsh;"

Category_A = Category_All 

**Category参数含义说明：**

| category              | 含义       | category               | 含义     | category           | 含义     | category           | 含义     | category             | 含义           |
| --------------------- | ---------- | ---------------------- | -------- | ------------------ | -------- | ------------------ | -------- | -------------------- | -------------- |
| category_ndbg_szsh    | 年报       | category_qyfpxzcs_szsh | 权益分派 | category_gszl_szsh | 公司治理 | category_pg_szsh   | 配股     | category_gqbd_szsh   | 股权变动       |
| category_bndbg_szsh   | 半年报     | category_dshgg_szsh    | 董事会   | category_zj_szsh   | 中介报告 | category_jj_szsh   | 解禁     | category_bcgz_szsh   | 补充更正       |
| category_yjdbg_szsh   | 一季报     | category_jshgg_szsh    | 监事会   | category_sf_szsh   | 首发     | category_gszq_szsh | 公司债   | category_cqdq_szsh   | 澄清致歉       |
| category_sjdbg_szsh   | 三季报     | category_gddh_szsh     | 股东大会 | category_zf_szsh   | 增发     | category_kzzq_szsh | 可转债   | category_fxts_szsh   | 风险提示       |
| category_yjygjxz_szsh | 业绩预告   | category_rcjy_szsh     | 日常经营 | category_gqjl_szsh | 股权激励 | category_qtrz_szsh | 其他融资 | category_tbclts_szsh | 特别处理和退市 |
| category_tszlq_szsh   | 退市整理期 |                        |          |                    |          |                    |          |                      |                |

In [3]:
# 3、定义要爬取的时间段———必选项

SeDate_A = '2000-01-01~2024-07-11'

In [4]:
# 4、定义爬取指定报告的关键词（若不指定则保持为空即可）———可选项

Search_key = ''

> ---------------------------------------------------使用说明：以下代码非必要请勿修改！-------------------------------------------------------

## Step2：导入工具包

In [5]:
import requests
# 用于获取网页内容
from urllib.request import urlretrieve
# 用于下载网络文件到本地
import re
# 用于正则匹配
import math
# 用于调用数学函数
import json
# 用于解析json
import xlwt
# 用于写入excel
import xlrd
# 用于读取excel
import os
# 用于实现系统功能
import pandas as pd
# 导入pandas工具库

# 若此处报错 ModuleNotFoundError:No module named 'XXX' 时说明XXX包未安装，需自行在控制台通过pip或conda进行安装（小白自行百度“python安装函数包”）

## Step3：获取巨潮资讯的数据源格式信息

In [6]:
# 从以下url中提取所需要的上市企业数据源信息（该url包括了巨潮资讯网上目前所有上市企业的stockList信息，我们的目的是从中获取到不同股票的orgId，以便接下来通过orgId去爬取不同企业的指定报告）
url = "http://www.cninfo.com.cn/new/data/szse_stock.json"
ret = requests.get(url=url)
ret = ret.content
stock_list = json.loads(ret)["stockList"]

# 查看巨潮资讯网一共收录了多少家企业数据
len(stock_list)

5720

In [7]:
# 输出stock_list中的前两项，查看一下列表中的具体数据形式
print(stock_list[:2])

[{'code': '000001', 'pinyin': 'payh', 'category': 'A股', 'orgId': 'gssz0000001', 'zwjc': '平安银行'}, {'code': '000002', 'pinyin': 'wka', 'category': 'A股', 'orgId': 'gssz0000002', 'zwjc': '万科A'}]


## Step4：用pandas把stock_list信息导出为本地Excel文件以便日后备用（可选）

In [8]:
def export_excel(export):
    # 将字典列表转换为DataFrame
    pf = pd.DataFrame(export)
    # 指定字段顺序
    order = ['orgId', 'category', 'code', 'pinyin', 'zwjc']
    pf = pf[order]
    # 将列名替换为中文
    columns_map = {
        'orgId': 'orgId（原始ID）',
        'category': 'category（股市类型）',
        'code': 'code（代码）',
        'pinyin': 'pinyin（拼音）',
        'zwjc': 'zwjc（）'
    }
    pf.rename(columns=columns_map, inplace=True)
    # 指定stock_list的存储路径（此处可自行更改）
    file_path = pd.ExcelWriter(Stocklist_path)
    # 替换空单元格
    pf.fillna(' ', inplace=True)
    # 输出
    pf.to_excel(file_path, encoding='utf-8', index=False)
    # 保存表格
    file_path.save()


if __name__ == '__main__':
    # 将分析完成的列表导出为excel表格
    export_excel(stock_list)

C:\Users\qq264\AppData\Local\Temp\ipykernel_134220\827394881.py:17: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  file_path = pd.ExcelWriter(Stocklist_path)


## Step5：从stock_list中提取code与orgId建立“code-orgId”匹配清单

In [9]:
# 提取stock_list中的code与orgId，遍历生成独立字典
code_dic = {(it['code']): it['orgId'] for it in stock_list}

print("\n正在进行服务器数据库检索，共找到 {} 家上市公司！\n\n --------------------------------服务器数据比对工作完成，即将开始载入数据进行匹配！--------------------------------\n".format(len(code_dic)))

# 若要输出查看具体“code-orgId”匹配内容请取消注释此行
# print(code_dic)

print("\n --------------------------数据匹配完成，共生成 {} 家上市公司的“code-orgId”匹配数据！--------------------------\n".format(len(code_dic)))




正在进行服务器数据库检索，共找到 5720 家上市公司！

 --------------------------------服务器数据比对工作完成，即将开始载入数据进行匹配！--------------------------------


 --------------------------数据匹配完成，共生成 5720 家上市公司的“code-orgId”匹配数据！--------------------------



## Step6：从Excel文件读取待爬取的股票清单并生成列表code_list

In [10]:
# 定义一个读取xls文件数据并转为列表的类

class excel_read:
    def __init__(self, excel_path=Codelist_path, encoding='utf-8', index=0):  # 待爬取企业清单路径
        #       获取文本对象
        self.data = xlrd.open_workbook(excel_path)
#       根据index获取某个sheet
        self.table = self.data.sheets()[index]
#       获取当前sheet页面的总行数,把每一行数据作为list放到 list
        self.rows = self.table.nrows

    def get_data(self):
        result = []
        for i in range(self.rows):
            #           获取每一行的数据
            col = self.table.row_values(i)
            print(col)
            result.append(col)
        print("待爬取企业已从xls文件中加载完毕，结果如下：\n {}\n".format(result))
        return result

# 运用函数生成待爬取企业的code_list
code_list = []
code_list.extend(excel_read().get_data())

['000656', '*ST金科']
待爬取企业已从xls文件中加载完毕，结果如下：
 [['000656', '*ST金科']]



# Part2：正式工作

## Step1：定义爬取函数

In [11]:
# 1、对单个页面进行请求，并返回数据信息——通过data自定义特定企业
def get_and_download_pdf_flie(pageNum, stock, searchkey='', category='', seDate=''):
    url = 'http://www.cninfo.com.cn/new/hisAnnouncement/query'
    pageNum = int(pageNum)
#   定义表单数据
    data = {'pageNum': pageNum,
            'pageSize': 30,
            'column': '',
            'tabName': 'fulltext',
            'plate': '',
            'stock': stock,
            'searchkey': searchkey,
            'secid': '',
            'category': category,
            'trade': '',
            'seDate': seDate,
            'sortName': '',
            'sortType': '',
            'isHLtitle': 'true'}

#   定义请求头（非小白建议更改为自己浏览器的headers）
    headers = {'Accept': '*/*',
               'Accept-Encoding': 'gzip, deflate',
               'Accept-Language': 'zh-CN,zh;q=0.9',
               'Connection': 'keep-alive',
               'Content-Length': '181',
               'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
               'Host': 'www.cninfo.com.cn',
               'Origin': 'http://www.cninfo.com.cn',
               'Referer': 'http://www.cninfo.com.cn/new/commonUrl/pageOfSearch?url=disclosure/list/search',
               'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
               'X-Requested-With': 'XMLHttpRequest'}

#   提交请求
    r = requests.post(url, data=data, headers=headers)

#   获取单页年报的数据，数据格式为json，解析并获取json中的年报信息
    result = r.json()['announcements']


# 2.对数据信息进行提取
    for i in result:
        #         避免下载一些年报摘要等不需要的文件
        if re.search('摘要', i['announcementTitle']):
            pass
        else:
            title = i['announcementTitle']

#           获取公告文件名
            secName = i['secName']

#           获取公司股票代码
            secCode = i['secCode']

#           获取adjunctUrl，并组合生成pdf文件下载地址（分析得知巨潮资讯数据库pdf下载地址格式：http://static.cninfo.com.cn/+adjunctUrl）
            adjunctUrl = i['adjunctUrl']
            down_url = 'http://static.cninfo.com.cn/'+adjunctUrl

#           定义下载之后需要保存到本地的文件名
            filename = f'{secCode}{secName}{title}.pdf'
    
#           用正则表达式将公告文件名中的特殊符号去掉，因为保存文件时命名规则不能带有某些特殊符号（比如*号），否则会导致程序报错
            filename = re.sub(r'[(*)(<)(>)(\\)(|)(\")(?)(:)(/)(<em>)(</em>)]', '', filename)

#           定义文件存放地址
            filepath = saving_path+'\\'+filename

#           提交下载请求
            r = requests.get(down_url)

#           用response.content来写入文件信息
            with open(filepath, 'wb') as f:
                f.write(r.content)

#           设置进度条
            print(f'{filename}下载完毕')

## Step2：自定义保存路径及确定待爬企业数量

In [12]:
# 设置存储年报文件的具体路劲
saving_path = Output_path

# 根据code_list计算待爬企业数量
Num = len(code_list)

print("待爬取企业总数量为：{}\n\n ------------------------------------已加载待爬取企业匹配信息，即将开始爬取数据！------------------------------------\n".format(Num))

待爬取企业总数量为：1

 ------------------------------------已加载待爬取企业匹配信息，即将开始爬取数据！------------------------------------



## Step3：设定参数进行遍历爬取

In [14]:
# 从code_list中根据待爬企业数量遍历提取code与orgIdS
for i in range(0, Num):
    code = code_list[i][0]
    orgId = code_dic[code]
#     定义stock
    stock = '{},{}'.format(code, orgId)
    print("\n即将爬取 {} ：".format(stock))
#     定义searchkey
    searchkey_A = Search_key
#     定义category
    category = Category_A
#     定义seDate
    seDate = SeDate_A

#     定义pageNum（需要通过观测，确保pageNum能够涵盖每一次遍历请求的结果页数，此处定为2页即可）
    for pageNum in range(8, 10):
        try:
            get_and_download_pdf_flie(
                pageNum, stock, searchkey_A, category, seDate,)
        except:
            #        超出页数后会报错，需要跳过异常以便于继续遍历执行
            pass

print("-------------------------------------------------程序执行完成！-------------------------------------------------")


即将爬取 000656,gssz0000656 ：
000656金科股份关于存续债务融资工具相关事项的公告.pdf下载完毕
000656金科股份关于发行股份购买资产暨关联交易事项的进展公告.pdf下载完毕
000656金科股份公司累计诉讼、仲裁事项的公告.pdf下载完毕
000656金科股份关于对参股公司提供担保的进展公告.pdf下载完毕
000656金科股份关于公司被债权人申请重整的进展公告.pdf下载完毕
000656金科股份关于对控股子公司提供担保的进展公告.pdf下载完毕
000656金科股份关于股票交易异常波动的公告.pdf下载完毕
000656金科股份关于股票交易异常波动的公告.pdf下载完毕
000656金科股份2023年半年度业绩预告.pdf下载完毕
000656金科股份关于卓越共赢计划暨2019至2023年员工持股计划之一期持股计划存续期即将届满的提示性公告.pdf下载完毕
000656金科股份关于股票交易异常波动的公告.pdf下载完毕
000656金科股份关于存续期公司债券及资产证券化产品展期完成的公告.pdf下载完毕
000656金科股份关于公司被债权人申请重整的进展公告.pdf下载完毕
000656金科股份关于签署战略投资框架协议的公告.pdf下载完毕
000656金科股份关于深圳证券交易所问询函回复的公告.pdf下载完毕
000656金科股份金科股份发行股份购买资产暨关联交易预案（修订稿）.pdf下载完毕
000656金科股份关于公司债务情况的公告.pdf下载完毕
000656金科股份公司累计诉讼、仲裁事项的公告.pdf下载完毕
000656金科股份北京市中伦（重庆）律师事务所关于金科地产集团股份有限公司2023年第四次临时股东大会的法律意见书.pdf下载完毕
000656金科股份关于对参股公司提供担保的进展公告.pdf下载完毕
000656金科股份关于2023年第四次临时股东大会决议公告.pdf下载完毕
000656金科股份关于对控股子公司提供担保的进展公告.pdf下载完毕
000656金科股份金科股份发行股份购买资产暨关联交易预案.pdf下载完毕
000656金科股份董事会关于本次重组符合《上市公司监管指引第9号——上市公司筹划和实施重大资产重组的监管要求》第四条规定的说明.pdf下载完毕
000656金科股份关于第十一届董事会第三十